# A spam filter based on the Naive Bayes algorithm

Spam messages are a nuissance to consumers, and effective spam filtering represents a real advantage for any service provider. In this project, a dataset of text messages will be used to build a basic spam filter based on the Naive Bayes algorithm.

Spam filtering represents a binary classification problem, because messages should either be spam or ham. Bayes theorem allows us to quantify the intensity of a hypothesis based on the observed data like so:
<br>![Bayes.png](Bayes.png)

Applying Bayes theorem to our problem, we would need to calculate the following items for both spam and ham messages:
- The prior, *P*(H). In our case, the frequencies of spam and ham, respectively.
- The likelihood, *P*(D | H). In our case, how often each word (D) occurs relative to all the words in spam or ham messages (H), respectively. 
- *P*(D). In our case, how frequently each word  occurs relative to the words in all messages.

Most messages, however, have several words. Following the basic rules of probability, we can obtain the posterior probability of class y (i.e. ham or spam) given features x<sub>1</sub> to x<sub>n</sub> (i.e. words in the message) like so:
<br>![Bayes_expanded.png](Bayes_expanded.png)

Since the denominator is the same, no matter whether we are trying to calculate P(spam | message) or P(ham | message), we can drop it. Instead, we can rely on proportionality: 
<br>![Bayes_proportional.png](Bayes_proportional.png)

To determine whether a message is spam or ham, we finally need to compare P(spam | message) and P(ham | message). The class with the higher probability should be our output.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Exploration

In [2]:
# Loading data

msgs = pd.read_csv('data/SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

In [3]:
msgs.shape

(5572, 2)

In [4]:
msgs.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# Frequencies of ham and spam

msgs['Label'].value_counts(normalize = True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

## Randomization & Train/Test split

In [6]:
# Randomizing messages

msgs_random = msgs.sample(frac=1, random_state=1)

In [7]:
# Selecting ~80% of the data for training, ~20% for testing

split_index = round(len(msgs_random) * 0.8)
msgs_train = msgs_random[:split_index].reset_index(drop=True)
msgs_test = msgs_random[split_index:].reset_index(drop=True)

After the train/test split, we should have very similar frequencies of ham and spam in both sets. Otherwise our model may not translate well.

In [8]:
# Checking frequencies in train

msgs_train['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [9]:
# Checking frequencies in test

msgs_test['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

## Data cleaning

To simplify our calculations, we will assume that there is no difference between uppercase and lowercase words. We will now:
- convert all words to lowercase
- replace non-word characters with a space. 

In [10]:
msgs_train_cln = msgs_train.copy()
msgs_train_cln['SMS'] = msgs_train_cln['SMS'].str.replace('\W', ' ')
msgs_train_cln['SMS'] = msgs_train_cln['SMS'].str.lower()
msgs_train_cln.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## Tokenizing messages

The first step is to obtain a list of unique words in the entire dataset.

In [11]:
# Converting messages from string to list

msgs_train_cln['SMS'] = msgs_train_cln['SMS'].str.split()

In [12]:
# Extracting unique words

vocabulary = []
for row in msgs_train_cln['SMS']:
    for item in row:
        vocabulary.append(item)

vocabulary = list(set(vocabulary))

len(vocabulary)

7783

The next step is to create a dataframe with messages as rows and words as columns. If a given word appears in a message, the column for that word will have a value of 1 in that rows. For words that do not appear in that message, the corresponding columns will have a value of 0.

In [13]:
# Creating an all-zero dictionary with words as keys
word_counts_per_sms = {word: [0] * len(msgs_train_cln['SMS']) for word in vocabulary}

# Populating from dataframe
for index, sms in enumerate(msgs_train_cln['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
# Converting dictionary to dataframe
word_count_df = pd.DataFrame(word_counts_per_sms)

# Concatenating original df with tokenized df
final_training = pd.concat([msgs_train_cln, word_count_df], axis=1)
final_training

,Label,SMS,series,125gift,attach,oxygen,fifteen,yuou,prof,swimsuit,...,prepared,eng,unkempt,den,wipro,match,tmrw,gonnamissu,surly,poortiyagi
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,ham,"[ok, i, thk, i, got, it, then, u, wan, me, 2, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,ham,"[i, want, kfc, its, tuesday, only, buy, 2, mea...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ham,"[no, dear, i, was, sleeping, p]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,ham,"[ok, pa, nothing, problem]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,ham,"[ill, be, there, on, lt, gt, ok]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating the priors

I.e. the frequencies of spam and ham.

In [14]:
proportions = final_training['Label'].value_counts(normalize = True)
p_spam = proportions['spam']
p_ham = proportions['ham']

## Calculating likelihoods

I.e. the frequency of each word divided by all the words in spam or ham multiplied by all the possible words.

P(w | spam) = (N<sub>w_spam</sub> + alpha) / (N<sub>spam</sub> + alpha * N<sub>vocabulary</sub>)<br>
<br>P(w | ham) = (N<sub>w_ham</sub> + alpha) / (N<sub>ham</sub> + alpha * N<sub>vocabulary</sub>)

We will apply Laplace smoothing, so alpha = 1

In [15]:
# Number of words in all spam messages
train_spam = final_training[final_training['Label'] == 'spam']
spam_msg_len = train_spam['SMS'].apply(len)
n_spam = spam_msg_len.sum()

# Number of words in all ham messages
train_ham = final_training[final_training['Label'] == 'ham']
ham_msg_len = train_ham['SMS'].apply(len)
n_ham = ham_msg_len.sum()

# Alpha for laplace smoothing
alpha = 1

print(p_spam, p_ham, n_spam, n_ham)

0.13458950201884254 0.8654104979811574 15190 57237


In [16]:
# Populating dictionaries with p_given_spam and p_given_ham for each unique word.

p_given_spam = {}
p_given_ham = {}

for word in vocabulary:
    p_given_spam[word] = (train_spam[word].sum() + alpha) / (n_spam + alpha * len(vocabulary))
    p_given_ham[word] = (train_ham[word].sum() + alpha) / (n_ham + alpha * len(vocabulary))

In [17]:
# Printing some examples 

print(p_given_spam['free'])
print(p_given_ham['free'])
print(p_given_spam['love'])
print(p_given_ham['love'])

0.00766116745745005
0.0007536142725315288
0.0004352936055369347
0.002522300830513688


## Building the classifier

To classify unseen new messages, we need a function which does several things:
- process new messages like the ones in our training set,
- use the appripriate priors and likelihoods to calculate unnormalised *P*(ham) and *P*(spam),
- compare the two,
- and finally return the class with higher probability.

If there is a tie between the two, our function should alert us so we can classify it manually.

In [18]:
import re

def classify(message):
    """Determines whether it a text message is spam or ham.
    Args:
        message (str): string containing the message
    Returns:
        classified (str): 'ham', 'spam' or 'needs human classification'
    """
    # Process message, i.e. replace all non-word characters with a space,
    # split message into a list of strings.
    message = re.sub('\W', ' ', message).lower().split()
   
    # Calculate unnormalised posteriors
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_given_spam:
            p_spam_given_message *= p_given_spam[word]
        if word in p_given_ham:
            p_ham_given_message *= p_given_ham[word]
    
    # Compare the posteriors and return class
    if p_ham_given_message > p_spam_given_message:
        classified = 'ham'
    elif p_ham_given_message < p_spam_given_message:
        classified = 'spam'
    else:
        classified = 'needs human classification'
    return classified

In [19]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

'spam'

In [20]:
classify("Sounds good, Tom, then see u there")

'ham'

## Evaluating the classifier

In [21]:
msgs_test['prediction'] = msgs_test['SMS'].apply(classify)